## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path_s = [path.curdir, "TrustZone_Flash-S"]
project_path_s

['.', 'TrustZone_Flash-S']

In [6]:
project_path_ns = [path.curdir, "TrustZone_Flash-NS"]
project_path_ns

['.', 'TrustZone_Flash-NS']

In [7]:
atprogram(path.abspath(path.join(*project_path_ns)), verbose=2)
atprogram(path.abspath(path.join(*project_path_s)), verbose=2, erase=False)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:41578
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00008000. Size = 0x000006d4.
[DEBUG] Memory segment base written at 0x000086d4. Size = 0x00000064.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "program"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:41584
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Memory segment base written a

0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [10]:
config_dict = {
    "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "gpio_delay_time" : 0.0007,
    "file_name_base": "experiment_trustzone_flash"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "plot_pins_method": "line",
    "plot_xmax": 1,
    "window_title": "Experiment TrustZone Flash",
}

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Perform the measurement.

In [11]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
    
with DGILibExtra(**cd) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger.log(500,stop_fn)
    data = dgilib.data

In [12]:
print(data)

Interfaces:
	 256:   power,   samples: 7575000
	  48:   gpio,    samples:   32674



## Analysis

Create Stop Function to stop parsing the data when all pins are high.

In [13]:
def stop_function(pin_values):
    return all(pin_values)

Parse the data.

In [14]:
nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)

In [16]:
nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

In [17]:
MIN_NUM_BYTES = 1
num_bytes = range(MIN_NUM_BYTES, MIN_NUM_BYTES + len(nsc_store_charge))
print(f"MAX_NUM_BYTES: {MIN_NUM_BYTES + len(nsc_store_charge) - 1}")

MAX_NUM_BYTES: 4084


In [18]:
from lmfit import Model

def line(x, slope, intercept):
    """a line"""
    return [slope*i + intercept for i in x]

mod = Model(line)
pars = mod.make_params(slope=0, intercept=1)

In [19]:
results = []
for y in [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]:
    result = mod.fit(y, pars, x=num_bytes)
    print(result.fit_report())
    results.append(result)

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 4084
    # variables        = 2
    chi-square         = 4.7956e-09
    reduced chi-square = 1.1748e-12
    Akaike info crit   = -112185.129
    Bayesian info crit = -112172.499
[[Variables]]
    slope:      2.4610e-08 +/- 1.4386e-11 (0.06%) (init = 0)
    intercept:  3.1263e-06 +/- 3.3927e-08 (1.09%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(slope, intercept) = -0.866

[[Model]]
    Model(line)
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 7
    # data points      = 4084
    # variables        = 2
    chi-square         = 6.4891e-13
    reduced chi-square = 1.5897e-16
    Akaike info crit   = -148565.036
    Bayesian info crit = -148552.407
[[Variables]]
    slope:      7.9483e-10 +/- 1.6735e-13 (0.02%) (init = 0)
    intercept:  1.8307e-08 +/- 3.9466e-10 (2.16%) (init = 1)
[[Correlations]] (unrep

In [20]:
fig2 = plt.figure(figsize=(9, 8))
fig2.canvas.set_window_title("Analysis TrustZone Flash")

In [21]:
charge_color = 'r'
time_color = 'b'

In [22]:
fig2.clf()
# fig2.suptitle("Energy analysis of AES")
ax1 = fig2.add_subplot(1, 1, 1)
ax2 = ax1.twinx()
ax1.set_xlabel('Number of bytes')
ax1.set_ylabel('Charge [C]', color=charge_color)
ax2.set_ylabel('Time [s]', color=time_color)
ax1.tick_params('y', colors=charge_color)
ax2.tick_params('y', colors=time_color)

In [23]:
lines = []
lines += ax1.plot(num_bytes, nsc_store_charge, charge_color+'-', label='TrustZone Flash Store Charge')
lines += ax1.plot(num_bytes, nsc_load_charge, charge_color+'--', label='TrustZone Flash Load Charge')
lines += ax2.plot(num_bytes, nsc_store_time, time_color+'-', label='TrustZone Flash Store Time')
lines += ax2.plot(num_bytes, nsc_load_time, time_color+'--', label='TrustZone Flash Load Time')
ax1.legend(handles=lines)
# [nsc_store_charge, nsc_load_charge, nsc_store_time, nsc_load_time]
ax1.set_title(f"TrustZone Flash Store Charge: Base {results[0].params['intercept'].value:.03} C plus {results[0].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Flash Load Charge: Base {results[1].params['intercept'].value:.03} C plus {results[1].params['slope'].value:.03} mC per byte\n" +
             f"TrustZone Flash Store Time: Base {results[2].params['intercept'].value:.03} s plus {results[2].params['slope'].value:.03} s per byte\n" +
             f"TrustZone Flash Load Time: Base {results[3].params['intercept'].value:.03} s plus {results[3].params['slope'].value:.03} s per byte\n")
fig2.tight_layout()
fig2.show()

In [25]:
# Save Charge amount list into pickle file
import pickle
pickle.dump(nsc_store_charge, open("trustzone_flash_store_charge.p", "wb"))
pickle.dump(nsc_load_charge, open("trustzone_flash_load_charge.p", "wb"))

## Write config file

In [11]:
import json

config = {}
config["name"] = "TrustZone Flash"
config["project_paths"] = [project_path_ns, project_path_s]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["TrustZone Flash Store"], 3: ["TrustZone Flash Load"]}, 
                      "result_types": ["Charge", "Time"],
                      "labels": {
                          "Charge": {"x":"Data Size", "x_unit": "bytes", "y": "Charge", "y_unit": "C"},
                          "Time": {"x":"Data Size", "x_unit": "bytes", "y": "Time", "y_unit": "s"},
                      },
                      "x_step": 1}
config["stop_functions"] = {"log": "def log_stop_function(logger_data): return all(logger_data.gpio.values[-1])",
                           "analysis": "def analysis_stop_function(pin_values): return all(pin_values)"}

with open("looped_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

# Python script to do measurement

In [26]:
%%writefile experiment_trustzone_flash.py

from os import getcwd, path
import pickle

from pydgilib_extra import *
from atprogram.atprogram import atprogram

def experiment_trustzone_flash(project_root=getcwd(), verbose=1):
    project_path_s = path.join(project_root, "TrustZone_Flash-S")
    project_path_ns = path.join(project_root, "TrustZone_Flash-NS")
    atprogram(project_path_ns, verbose=verbose)
    atprogram(project_path_s, verbose=verbose, erase=False)

    config_dict = {
        "interfaces": [INTERFACE_POWER, INTERFACE_GPIO],
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_trustzone_flash",
        "log_folder": project_root,
        "verbose": verbose-1
    }
    
    def stop_fn(logger_data):
        return all(logger_data.gpio.values[-1])
    
    if verbose:
        print(f"Start DGILibExtra with config: \n{config_dict}\n")

    data = []
    with DGILibExtra(**config_dict) as dgilib:
        dgilib.device_reset()
        dgilib.logger.log(500,stop_fn)
        data = dgilib.data
        
    if verbose:
        print(f"DGILibExtra data: {data}")

    def stop_function(pin_values):
        return all(pin_values)

    nsc_store_charge, nsc_store_time = power_and_time_per_pulse(data, 2, stop_function=stop_function)
    nsc_load_charge, nsc_load_time = power_and_time_per_pulse(data, 3, stop_function=stop_function)

    # Save Charge amount list into pickle file
    pickle.dump(nsc_store_charge, open(path.join(project_root, "trustzone_flash_store_charge.p"), "wb"))
    pickle.dump(nsc_load_charge, open(path.join(project_root, "trustzone_flash_load_charge.p"), "wb"))
    
    if verbose:
        print(f"Saved results in: {path.join(project_root)}")

if __name__ == "__main__":
   experiment_trustzone_flash()

Overwriting experiment_trustzone_flash.py
